# [2023 임베디드시스템특강: 딥러닝]

# [과제3:] Regularization

딥러닝 모델의 용량에 비해서 학습데이터가 충분하지 않거나, 학습데이터에 노이즈가 많은 경우 **overfitting**(과적합)현상이 발생합니다. 

Overfitting은 training 데이터에서는 모델이 잘 동작하지만, 처음보는 데이터로 일반화(**generalization**)가 되지 않는 경우입니다. 

수업시간에 강조한대로, 머신/딥러닝의 목적은 학습데이터가 아닌, 처음보는 데이터에서 잘 동작하도록 만드는 것임을 기억하세요. 

본 과제에서는 overfitting현상을 감소시키는 regularization 기법들을 학습합니다. 


**배울 내용들:**
- L2 regularization
- Dropout


# 0 - 시작하기전에...
## 저 조영욱 는 본 과제를 수행하면서 다른사람의 도움을 받거나 주지 않았습니다. 

다음 패키지들을 import합니다. 

In [ ]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
from reg_utils import sigmoid, relu, plot_decision_boundary, initialize_parameters, load_2D_dataset, predict_dec
from reg_utils import compute_cost, predict, forward_propagation, backward_propagation, update_parameters
import sklearn
import sklearn.datasets
import scipy.io
from testCases import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (7.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

**Problem Statement**: 

본 문제에서는 프랑스 축구 대표팀에 딥러닝 인공지능을 도입한다고 가정합니다.
 
우리가 개발할 딥러닝 모델은 프랑스 골키퍼가 킥을 했을때 볼을 받은 선수가 한번의 터치만으로 골을 넣을 수 있는 위치를 추천해주는 모델입니다.  
<!-- You have just been hired as an AI expert by the French Football Corporation. They would like you to recommend positions where France's goal keeper should kick the ball so that the French team's players can then hit it with their head.  -->

<img src="images/field_kiank.png" style="width:600px;height:350px;">

위 그림은 축구 경기장입니다. 프랑스 골키퍼가 킥을 하면, 양 팀 선수들이 볼을 잡기위해서 경쟁합니다.

In [ ]:
train_X, train_Y, test_X, test_Y = load_2D_dataset()

위의 그래프는 지난 10경기에서 프랑스 골키퍼의 킥이 떨어진 위치입니다. 해당 위치에서부터 한번의 터치로 골을 넣었는지는 파란/빨강 색으로 표시합니다.
- 파란 점들은 프랑스 선수가 한번의 터치로 골을 넣었던 위치입니다.
- 빨간 점들은 프랑스 선수가 볼을 빼앗기거나, 한번의 터치로 골을 넣지 못한 위치입니다.

우리는 위의 학습데이터를 이용해서, 골키퍼가 어느 위치로 킥을 해야 프랑스 선수가 한번의 볼터치만으로 골을 넣을 수 있는지를 예측하도록 딥러닝 모델을 학습시켜야합니다.

그림상의 학습데이터를 보면, 데이터에 노이즈 (또는, 일반적이지 않은) 데이터가 있는것을 알 수 있습니다. 

노이즈에 지나치게 민감한 모델은 overfitting 현상이 나타날 수 있습니다.

Regularization이 적용되지 않은 경우와 적용된 경우에 이러한 노이즈가 학습에 어떤 영향을 주는지 살펴보도록 합니다.

## 1 - Non-regularized model

다음의 신경망 네트워크를 살펴보세요. 이미 모두 구현은 되어 있습니다. 
Regularization을 위해서 2개의 인자가 추가되었습니다. 
- *lmbd*: `lambd`는 L2 regularization의 정도를 조절하는 하이퍼파라미터입니다. lbmd=0일 경우, L2 regularization을 하지 않는 것입니다. 
- *keep_prob*: dropout에서 얼마만큼의 뉴런을 drop하지 않는지를 표시합니다. keep_prob=1일 경우, dropout을 적용하지 않는것과 같습니다.

먼저, L2 regularization과 dropout을 적용하지 않고 모델을 학습시켜봅니다. 그런다음, 다음의 함수를 구현하여 regularization을 적용시켜봅니다.
- *L2 regularization*은 다음의 두 함수에 구현됩니다: "`compute_cost_with_regularization()`" and "`backward_propagation_with_regularization()`"
- *Dropout* 은 다음의 두 함수에 구현됩니다: "`forward_propagation_with_dropout()`" and "`backward_propagation_with_dropout()`"

아래 함수는 과제에서 계속 사용되므로, 어떤일들을 하는지 살펴보도록 합니다.

In [ ]:
def model(X, Y, learning_rate = 0.3, num_iterations = 30000, print_cost = True, lambd = 0, keep_prob = 1):
    """
    Implements a three-layer neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SIGMOID.
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (output size, number of examples)
    learning_rate -- learning rate of the optimization
    num_iterations -- number of iterations of the optimization loop
    print_cost -- If True, print the cost every 10000 iterations
    lambd -- regularization hyperparameter, scalar
    keep_prob - probability of keeping a neuron active during drop-out, scalar.
    
    Returns:
    parameters -- parameters learned by the model. They can then be used to predict.
    """
        
    grads = {}
    costs = []                            # to keep track of the cost
    m = X.shape[1]                        # number of examples
    layers_dims = [X.shape[0], 20, 3, 1]
    
    # Initialize parameters dictionary.
    parameters = initialize_parameters(layers_dims)

    # Loop (gradient descent)

    for i in range(0, num_iterations):

        # Forward propagation: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID.
        if keep_prob == 1:
            a3, cache = forward_propagation(X, parameters)
        elif keep_prob < 1:
            a3, cache = forward_propagation_with_dropout(X, parameters, keep_prob)
        
        # Cost function
        if lambd == 0:
            cost = compute_cost(a3, Y)
        else:
            cost = compute_cost_with_regularization(a3, Y, parameters, lambd)
            
        # Backward propagation.
        assert(lambd==0 or keep_prob==1)    # it is possible to use both L2 regularization and dropout, 
                                            # but this assignment will only explore one at a time
        if lambd == 0 and keep_prob == 1:
            grads = backward_propagation(X, Y, cache)
        elif lambd != 0:
            grads = backward_propagation_with_regularization(X, Y, cache, lambd)
        elif keep_prob < 1:
            grads = backward_propagation_with_dropout(X, Y, cache, keep_prob)
        
        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Print the loss every 10000 iterations
        if print_cost and i % 10000 == 0:
            print("Cost after iteration {}: {}".format(i, cost))
        if print_cost and i % 1000 == 0:
            costs.append(cost)
    
    # plot the cost
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('iterations (x1,000)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

Regularization없이 학습을 합니다. 

train/test 데이터셋에서 분류정확도(accuracy)가 어떻게 차이가 나는지 확인합니다.

In [ ]:
parameters = model(train_X, train_Y)
print ("On the training set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

train accuracy는 94.8%이고, test accuracy는 91.5%로 차이가 납니다. 

Regularization을 적용하면 이러한 train/test accuracy의 차이를 줄일수 있어야 합니다.

Regularization없이 학습된 모델이 어떠한 경계를 만들어서 분류하는지 아래 코드로 확인합니다.


In [ ]:
plt.title("Model without regularization")
axes = plt.gca()
axes.set_xlim([-0.75,0.40])
axes.set_ylim([-0.75,0.65])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

Regularization없이 학습된 모델이 예외적인 경우인 노이즈까지 학습한 것을 볼 수 있습니다.

이러한 상황을 모델이 overfittting되었다고 합니다. 

이러한 overfitting을 줄일 수 있는 2가지 방법인 L2 regularization과 dropout에 대해서 살펴봅니다.

## 2 - L2 Regularization

L2 regularization에서는 cost function에 L2항을 추가합니다. 
원래는 다음과 같은 cost function이였습니다.
$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small  y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} \tag{1}$$
L2 regularization항을 추가한 cost function은 다음과 같이 바뀝니다.
$$J_{regularized} = \small \underbrace{-\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} }_\text{cross-entropy cost} + \underbrace{\frac{1}{m} \frac{\lambda}{2} \sum\limits_l\sum\limits_k\sum\limits_j W_{k,j}^{[l]2} }_\text{L2 regularization cost} \tag{2}$$


**Exercise**: 아래의 `compute_cost_with_regularization()` 함수에서 (2)에 주어진 cost function을 구현합니다.  

$\sum\limits_k\sum\limits_j W_{k,j}^{[l]2}$를 구현하기 위해서는 다음을 사용합니다 :
```python
np.sum(np.square(Wl))
```
3개의 계층이 있으므로 $W^{[1]}$, $W^{[2]}$ and $W^{[3]}$ 모두에 대해서 적용해서 합쳐야 합니다. 
또한, $ \frac{1}{m} \frac{\lambda}{2} $를 앞에 곱하는 것도 잊으면 안됩니다.

In [ ]:
# GRADED FUNCTION: compute_cost_with_regularization

def compute_cost_with_regularization(A3, Y, parameters, lambd):
    """
    Implement the cost function with L2 regularization. See formula (2) above.
    
    Arguments:
    A3 -- post-activation, output of forward propagation, of shape (output size, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    parameters -- python dictionary containing parameters of the model
    
    Returns:
    cost - value of the regularized loss function (formula (2))
    """
    m = Y.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]
    
    cross_entropy_cost = compute_cost(A3, Y) # This gives you the cross-entropy part of the cost
    
    ### START CODE HERE ### (approx. 1 line)
     L2_regularization_cost = lambd / (2 * m) * (np.sum(np.square(W1)) + np.sum(np.square(W2)) + np.sum(np.square(W3)))
    ### END CODER HERE ###
    
    cost = cross_entropy_cost + L2_regularization_cost
    
    return cost

In [ ]:
A3, Y_assess, parameters = compute_cost_with_regularization_test_case()

print("cost = " + str(compute_cost_with_regularization(A3, Y_assess, parameters, lambd = 0.1)))

**Expected Output**: 

<table> 
    <tr>
    <td>
    cost
    </td>
        <td>
    1.78648594516
    </td>
    </tr>
</table> 

cost function을 변경하였으므로, backward propagation과정도 변경됩니다. 

**Exercise**: 새로운 cost function에 대한 dW1, dW2 and dW3를 구합니다. 각각의 W에 대해서 reguration항의 gradient인 다음을 더해주어야 합니다: ($\frac{d}{dW} ( \frac{1}{2}\frac{\lambda}{m}  W^2) = \frac{\lambda}{m} W$).


In [ ]:
# GRADED FUNCTION: backward_propagation_with_regularization

def backward_propagation_with_regularization(X, Y, cache, lambd):
    """
    Implements the backward propagation of our baseline model to which we added an L2 regularization.
    
    Arguments:
    X -- input dataset, of shape (input size, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    cache -- cache output from forward_propagation()
    lambd -- regularization hyperparameter, scalar
    
    Returns:
    gradients -- A dictionary with the gradients with respect to each parameter, activation and pre-activation variables
    """
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    
    ### START CODE HERE ### (approx. 1 line)
    
    ### END CODE HERE ###
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    ### START CODE HERE ### (approx. 1 line)
    dW2 = 1. / m * np.dot(dZ2, A1.T) + + (lambd / m) * W2
    ### END CODE HERE ###
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    ### START CODE HERE ### (approx. 1 line)
    dW1 = 1. / m * np.dot(dZ1, X.T) + + (lambd / m) * W1
    ### END CODE HERE ###
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2,
                 "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1, 
                 "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [ ]:
X_assess, Y_assess, cache = backward_propagation_with_regularization_test_case()

grads = backward_propagation_with_regularization(X_assess, Y_assess, cache, lambd = 0.7)
print ("dW1 = \n"+ str(grads["dW1"]))
print ("dW2 = \n"+ str(grads["dW2"]))
print ("dW3 = \n"+ str(grads["dW3"]))

**Expected Output**:

```
dW1 = 
[[-0.25604646  0.12298827 -0.28297129]
 [-0.17706303  0.34536094 -0.4410571 ]]
dW2 = 
[[ 0.79276486  0.85133918]
 [-0.0957219  -0.01720463]
 [-0.13100772 -0.03750433]]
dW3 = 
[[-1.77691347 -0.11832879 -0.09397446]]
```

모델을 L2 regularization을 이용해서 학습시킵니다. $(\lambda = 0.7)$을 인자로 전달하면,
-  `compute_cost` 대신에 `compute_cost_with_regularization`가 호출되고,
- `backward_propagation` 대신에 `backward_propagation_with_regularization`가 호출됩니다.

In [ ]:
parameters = model(train_X, train_Y, lambd = 0.7)
print ("On the train set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

train accuracy와 test accuracy의 차이가 많이 줄었습니다. 

모델의 분류 경계를 그림으로 확인해봅니다. 

In [ ]:
plt.title("Model with L2-regularization")
axes = plt.gca()
axes.set_xlim([-0.75,0.40])
axes.set_ylim([-0.75,0.65])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

- L2 regularization이 학습의 경계를 좀더 부드럽게 만든것을 확인할 수 있습니다. 
- $\lambda$는 하이퍼파라미터로 사용자가 정해주는 값입니다. 
- $\lambda$를 0.7 0.1, 0.01 등으로 수정해서 위의 코드를 다시 실행해 보면서 차이점을 알아보도록 하세요. 

## 3 - Dropout

**Dropout**은 딥러닝에서 자주 사용되는 또 다른 regularization 기법입니다. 

**Dropout**에서는 학습중에 일정 비율의 뉴런을 무작위적으로 동작하지 않도록 출력을 0으로 만듭니다.

이렇게 되면, 특정 뉴런에 의존하는 현상을 줄일 수 있습니다. 

### 3.1 - Forward propagation with dropout

**Exercise**: 
dropout이 적용된 forward propagation을 구현합니다.

지금 과제에서 3개 계층의 신경망을 사용하고 있는데, 1째와 2째 계층에만 dropout을 적용하고, 입력과 출력 계층에는 dropout을 적용하지 않습니다. 

**Instructions**:
1, 2 계층에서 몇개의 뉴런을 동작하지 않도록 하기위애서 다음의 4단계가 필요합니다.
첫번째 계층을 예로 설명합니다. 
1. $a^{[1]}$과 모양이 같은 $d^{[1]}$을 `np.random.rand()`를 이용해서 만듭니다. 이 함수는 0과 1 사이의 실수를 랜덤하게 만들어 줍니다. 

여러개의 examples를 동시에 처리하는 vectorized 된 구현을 위해서는,
$D^{[1]}$를 $A^{[1]}$의 모양과 같도록 만들어 줘야 합니다: $D^{[1]} = [d^{[1](1)}, d^{[1](2)}, ... ,d^{[1](m)}]$ 

2. $D^{[1]}$ 값들중에서 `keep_prob`보다 작은 값들은 1로, 그렇지 않은 값들은 0으로 만듦니다. 

keep_prob=0.8이라고 가정하면, 80%의 뉴런은 동작하고, 나머지 20%는 dropout 하겠다는 것입니다.  

**힌트:** 
참고로, X의 80%만 1로 만들고 나머지는 0으로 만들기 위한 코드는 다음과 같습니다:

`X = (X < keep_prob).astype(int)`  

`.astype(int)`는 boolean 값인 `True`와 `False`를 1과 0으로 명시적으로 타입변환하기 위한 것입니다. 

3. $A^{[1]} * D^{[1]}$의 연산결과를 $A^{[1]}$ 로 대체합니다. 이는 $D^{[1]}$를 mask로 사용하여, $A^{[1]}$ 의 값들을 선택적으로 0으로 만들기 위한 것입니다. 
4. $A^{[1]}$ 를 `keep_prob`으로 나누어서 dropout 전후에 activation의 평균값이 같도록 만들어 주는 것입니다. (이것을 inverted dropout이라고 부릅니다.)

In [ ]:
# GRADED FUNCTION: forward_propagation_with_dropout

def forward_propagation_with_dropout(X, parameters, keep_prob = 0.5):
     """
     Implements the forward propagation: LINEAR -> RELU + DROPOUT -> LINEAR -> RELU + DROPOUT -> LINEAR -> SIGMOID.

     Arguments:
     X -- input dataset, of shape (2, number of examples)
     parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape (20, 2)
                    b1 -- bias vector of shape (20, 1)
                    W2 -- weight matrix of shape (3, 20)
                    b2 -- bias vector of shape (3, 1)
                    W3 -- weight matrix of shape (1, 3)
                    b3 -- bias vector of shape (1, 1)
     keep_prob - probability of keeping a neuron active during drop-out, scalar

     Returns:
     A3 -- last activation value, output of the forward propagation, of shape (1,1)
     cache -- tuple, information stored for computing the backward propagation
     """

     np.random.seed(1)

     # retrieve parameters
     W1 = parameters["W1"]
     b1 = parameters["b1"]
     W2 = parameters["W2"]
     b2 = parameters["b2"]
     W3 = parameters["W3"]
     b3 = parameters["b3"]

     # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
     Z1 = np.dot(W1, X) + b1
     A1 = relu(Z1)
     ### START CODE HERE ### (approx. 4 lines)         # Steps 1-4 below correspond to the Steps 1-4 described above. 
       D1 = np.random.rand(A1.shape[0], A1.shape[1])  # Step 1: initialize matrix D1 = np.random.rand(..., ...)
    D1 = (D1 < keep_prob)  # Step 2: convert entries of D1 to 0 or 1 (using keep_prob as the threshold)
    A1 = np.multiply(A1, D1)  # Step 3: shut down some neurons of A1
    A1 = A1 / keep_prob  # Step 4: scale the value of neurons that haven't been shut down
     ### END CODE HERE ###
     Z2 = np.dot(W2, A1) + b2
     A2 = relu(Z2)
     ### START CODE HERE ### (approx. 4 lines)
    D2 = np.random.rand(A2.shape[0], A2.shape[1])  # Step 1: initialize matrix D2 = np.random.rand(..., ...)
    D2 = (D2 < keep_prob)  # Step 2: convert entries of D2 to 0 or 1 (using keep_prob as the threshold)
    A2 = np.multiply(D2, A2)  # Step 3: shut down some neurons of A2
    A2 = A2 / keep_prob  # Step 4: scale the value of neurons that haven't been shut down
     ### END CODE HERE ###
     Z3 = np.dot(W3, A2) + b3
     A3 = sigmoid(Z3)

     cache = (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3)

     return A3, cache

In [ ]:
X_assess, parameters = forward_propagation_with_dropout_test_case()

A3, cache = forward_propagation_with_dropout(X_assess, parameters, keep_prob = 0.7)
print ("A3 = " + str(A3))

**Expected Output**: 

<table> 
    <tr>
    <td>
    A3
    </td>
        <td>
    [[ 0.36974721  0.00305176  0.04565099  0.49683389  0.36974721]]
    </td>
    </tr>
</table> 

### 3.2 - Backward propagation with dropout

**Exercise**: 
dropout이 적용된 backward propagation을 구현합니다. 
3계층 신경망에서 1,2 계층에 dropout을 적용하였으므로, gradiet계산을 위해서 마스크인 $D^{[1]}$와 $D^{[2]}$는 cache에 저장되어 있습니다. 

**Instruction**:
dropout의 backpropagation은 다음의 2가지 단계로 진행됩니다. 
1. forward propagation에서 값을 0으로 만든 뉴런을 back propagation에서도 동일하게 0으로 만듭니다. 마스크인 $D^{[1]}$를 cache에 저장해 두었으므로, `dA1`에 $D^{[1]}$를 곱하면 됩니다. 
2. forward propagation에서 `A1`을 `keep_prob`로 나눈것 같이 `dA1`도 `keep_prob`으로 나누어 줍니다. 
미분에서 `A1`에 `keep_prob`을 곱하는 경우, 미분값인 `dA1`도 역시 `keep_prob`만큼 곱해줘야합니다. 

In [ ]:
# GRADED FUNCTION: backward_propagation_with_dropout

def backward_propagation_with_dropout(X, Y, cache, keep_prob):
    """
    Implements the backward propagation of our baseline model to which we added dropout.
    
    Arguments:
    X -- input dataset, of shape (2, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    cache -- cache output from forward_propagation_with_dropout()
    keep_prob - probability of keeping a neuron active during drop-out, scalar
    
    Returns:
    gradients -- A dictionary with the gradients with respect to each parameter, activation and pre-activation variables
    """
    
    m = X.shape[1]
    (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    dA2 = np.dot(W3.T, dZ3)
    ### START CODE HERE ### (≈ 2 lines of code)
        # Step 1: Apply mask D2 to shut down the same neurons as during the forward propagation
        # Step 2: Scale the value of neurons that haven't been shut down
    ### END CODE HERE ###
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T)
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    ### START CODE HERE ### (≈ 2 lines of code)
        # Step 1: Apply mask D1 to shut down the same neurons as during the forward propagation
        # Step 2: Scale the value of neurons that haven't been shut down
    ### END CODE HERE ###
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2,
                 "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1, 
                 "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [ ]:
X_assess, Y_assess, cache = backward_propagation_with_dropout_test_case()

gradients = backward_propagation_with_dropout(X_assess, Y_assess, cache, keep_prob = 0.8)

print ("dA1 = \n" + str(gradients["dA1"]))
print ("dA2 = \n" + str(gradients["dA2"]))

**Expected Output**: 

```
dA1 = 
[[ 0.36544439  0.         -0.00188233  0.         -0.17408748]
 [ 0.65515713  0.         -0.00337459  0.         -0.        ]]
dA2 = 
[[ 0.58180856  0.         -0.00299679  0.         -0.27715731]
 [ 0.          0.53159854 -0.          0.53159854 -0.34089673]
 [ 0.          0.         -0.00292733  0.         -0.        ]]
```

dropout 값을 `keep_prob = 0.86`로 설정하고 모델을 학습시킵니다. 이렇게 하면, 학습중에 14%의 뉴런은 제거하는 효과가 있습니다. 
아래 model()함수에서 dropout을 위해서 호출되는 함수가 다음처럼 변경됩니다. 
- `forward_propagation` 대신에 `forward_propagation_with_dropout`
- `backward_propagation` 대신에 `backward_propagation_with_dropout`가 호출됩니다. 

In [ ]:
parameters = model(train_X, train_Y, keep_prob = 0.86, learning_rate = 0.3)

print ("On the train set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

Dropout을 적용해서 test accuracy가 95% 늘어난 것을 확인합니다. 이전의 overfitting문제가 해결되었습니다. 

아래의 그림으로 분류 경계가 어떻게 바뀌었는지 확인해 봅니다. 

`keep_prob`값을 다른 값으로 해서 모델을 학습하고, 결과과 어떻게 바뀌는지 확인해봅니다. `keep_prob`이 너무 크거나 작으면 어떤 현상이 나타납니까?

In [ ]:
plt.title("Model with dropout")
axes = plt.gca()
axes.set_xlim([-0.75,0.40])
axes.set_ylim([-0.75,0.65])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

**Note**:
마지막으로, dropout은 training시에만 overfitting을 없애기 위해서 적용되고, 실제 test시에는 사용하지 않는다는 것을 기억해야 합니다.


## 4 - Conclusions

**모든 결과를 요약하면 다음과 같습니다**: 

<table> 
    <tr>
        <td>
        model
        </td>
        <td>
        train accuracy
        </td>
        <td>
        test accuracy
        </td>
    </tr>
        <td>
        3-layer NN without regularization
        </td>
        <td>
        95%
        </td>
        <td>
        91.5%
        </td>
    <tr>
        <td>
        3-layer NN with L2-regularization
        </td>
        <td>
        94%
        </td>
        <td>
        93%
        </td>
    </tr>
    <tr>
        <td>
        3-layer NN with dropout
        </td>
        <td>
        93%
        </td>
        <td>
        95%
        </td>
    </tr>
</table> 

training accuracy가 떨어진 것을 확인할 수 있습니다. 그렇지만 test accuracy는 더 높아졌습니다. 

regularization은 네트워크의 능력을 특정방향을 갖도록 제한하는 역할을 합니다. 그러므로, 아무 제약없이 학습하는 것에 비해서 training accuracy는 떨어질 수 있습니다. 

그렇지만, 이러한 제약이 test시에는 도움이 되는 것입니다. 